In [1]:
import sys

In [2]:
import tensorflow as tf
print("GPU disponibile:", tf.config.list_physical_devices('GPU'))

2025-03-07 21:27:35.067192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-07 21:27:35.067217: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


GPU disponibile: []


2025-03-07 21:27:36.805493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-07 21:27:36.808120: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-07 21:27:36.808172: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2025-03-07 21:27:36.808213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2025-03-07 21:27:36.808250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
sys.path.append("../preprocessing")

In [4]:
import sampling as samp
import normalizzation as norm

In [5]:
sys.path.append("../models")

In [6]:
from lstm_model import LSTMModel

In [7]:
import pandas as pd
import numpy as np

# Caricamento dei dati da CSV
X_train = pd.read_csv("../partitioning/npy/partitions_1/X_train.csv").values  
X_test = pd.read_csv("../partitioning/npy/partitions_1/X_test.csv").values
X_val = pd.read_csv("../partitioning/npy/partitions_1/X_val.csv").values

y_train = pd.read_csv("../partitioning/npy/partitions_1/y_train.csv").values
y_test = pd.read_csv("../partitioning/npy/partitions_1/y_test.csv").values
y_val = pd.read_csv("../partitioning/npy/partitions_1/y_val.csv").values

# (Facoltativo) Verificare le dimensioni dei dati
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")


X_train shape: (3583, 50), y_train shape: (3583, 1)
X_test shape: (768, 50), y_test shape: (768, 1)
X_val shape: (767, 50), y_val shape: (767, 1)


In [8]:
# Esempio di input_shape per dati sequenziali: (timesteps=100, features=10)
input_shape = (50, 1)
lstm_model = LSTMModel(input_shape)

# Mostra la struttura del modello
lstm_model.summary()

# Addestramento del modello
lstm_model.train(X_train, y_train, epochs=5, batch_size=32, validation_data = (X_val, y_val))

lstm_model.save_model("mio_modello_lstm.h5")
lstm_model.load_model("mio_modello_lstm.h5")

2025-03-07 21:27:36.972781: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            10400     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
112/112 [==============================] -